In [2]:
import csv
import re

def clean_content(text):
    """기사 본문에서 불필요한 출처, 특수문자, 영어 단어, 특정 문구 제거"""
    
    # 특정 문구 제거
    unwanted_phrases = [
        "세상의 모든 골 때리는 이야기",
        "파스 속보이는 연예뉴스 스타",
        "파이낸셜뉴스",
        "속보",
        "사진"  # '파이낸셜뉴스'가 포함된 문장 삭제
    ]
    
    # unwanted_phrases에 있는 문구들을 제거 (양쪽 공백도 제거)
    for phrase in unwanted_phrases:
        text = re.sub(r'\s*' + re.escape(phrase) + r'\s*', '', text)
    
    # 문장에서 '헉!'이 포함된 문장을 제거
    sentences = text.split('.')
    sentences = [sentence.strip() for sentence in sentences if '헉!' not in sentence]
    
    # '파이낸셜뉴스'가 포함된 문장도 삭제
    sentences = [sentence for sentence in sentences if "파이낸셜뉴스" not in sentence]
    sentences = [sentence for sentence in sentences if "연예뉴스" not in sentence]
    
    # 영어 단어 제거 (영어 알파벳만 포함된 단어 제거)
    sentences = [re.sub(r'[a-zA-Z]+', '', sentence) for sentence in sentences]
    
    # 남은 문장을 다시 합침
    text = '. '.join(sentences).strip()

    # 구두점 제외한 특수문자만 제거
    text = re.sub(r'[^\w\s\.,!?-]', '', text)  # 구두점(.,!?-)는 남기고 나머지 특수문자 제거

    # ". ." 사이의 빈칸 제거
    text = re.sub(r'\s*\.\s*\.', '.', text)  # . . 사이의 빈칸을 없앰
    
    # content 길이가 12 미만일 경우 NaN으로 처리
    if len(text) < 12:
        return None  # None은 NaN처럼 취급됩니다.
    
    return text

def process_csv_file(input_file, output_file):
    """CSV 파일을 처리하여 불필요한 문장과 영어 단어, 특정 문구 제거 후 저장"""
    with open(input_file, 'r', encoding='utf-8') as in_file:
        reader = csv.reader(in_file)
        header = next(reader)  # 헤더는 첫 번째 줄에 있음
        
        rows = []
        for row in reader:
            date, title, file_id, content = row
            
            # "속보"가 포함된 제목의 경우 content를 title로 설정
            if "속보" in title:
                content = title  # title과 content를 동일하게 설정

            # content가 None(NaN)인 경우 넘어가기
            if content is None or content.strip() == "":
                continue  # content가 없으면 건너뜀

            # content에서 '헉!'이 포함된 문장, 영어 단어, '파이낸셜뉴스' 문장, 특정 문구 제거
            content = clean_content(content)
            
            # 수정된 row 추가
            rows.append([date, title, file_id, content])
        
        # content가 NaN인 행 필터링
        rows = [row for row in rows if row[3] is not None]

    # 수정된 내용을 새 CSV 파일로 저장
    with open(output_file, 'w', encoding='utf-8', newline='') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(header)  # 헤더 작성
        writer.writerows(rows)  # 수정된 내용 기록

# CSV 파일 경로 설정
input_file = '파이낸셜 최종 클렌징.csv'  # 원본 CSV 파일
output_file = '파이낸셜_진짜_최종.csv'  # 수정된 CSV 파일

# CSV 파일 처리 함수 호출
process_csv_file(input_file, output_file)
